## 2.2 階層データと閉包性

概要の詳細はテキストを参照してください。  
ここでは重要と思われる内容について説明を記載します。

図2.2は、(cons 1 2)の、箱とポインタ記法の表現を表しています。  

<img src="2.2.png" width="75%">

<div style="text-align: center;">図2.2:(cons 1 2)の箱とポインタ表現</div>

consは数値だけでなくペアも結合できます。  
図2.3はペアを使って数値1,2,3,4を組わせる方法を示しています。  
ペアを要素とするペアを作る能力は、
consの**閉包性**(closure property)と呼ばれています。  

<img src="2.3.png" width="75%">

<div style="text-align: center;">図2.3:ペアを使って1, 2, 3, 4を組み合わせる⼆つの⽅法</div>

In [1]:
; ペアを要素として構築した結果1
(cons (cons 1 2) (cons 3 4))

((1 . 2) 3 . 4)

In [2]:
; ペアを要素として構築した結果2
(cons (cons 1 (cons 2 3)) 4)

((1 2 . 3) . 4)

※上記のようなペアの組み合わせは木として使用します。

この節では、
複合データの閉包がもたらす結果について扱い、  
ペアを使って列(リスト)や⽊を表現する確⽴されたテクニックについて説明します。

### 2.2.1 列の表現

#### 概要
以下は2.2.1で扱う内容です。  
- 列の表現
- リスト演算
    - cdrダウン
    - consアップ
    - list-ref
    - length (再帰プロセス)
    - length (反復プロセス)
    - append
    - 練習問題2.17 last-pair
    - 練習問題2.18 reverse
    - 練習問題2.19 リストを使った両替の組み合わせパターン
    - 練習問題2.20 ドット記法(可変引数)
- リストに対するマップ
    - 練習問題2.21 square-list mapの使用方法
    - 練習問題2.22 square-listの反復プロセス
    - 練習問題2.23 for-each

#### 列の表現

列(sequence)とは、データオブジ ェクトの順序つき集合です。  
ペアを使って列を表すやり⽅はいろいろありますが(キューとかも列の一部としている？)、  
ここではリストとよばれる、チェーンでペアを接続する表現方法について説明します。  

図2.4は、1,2,3,4のリストの表現方法を表しています。  
リスト(list)という⾔葉はリスト終端マーカー(nil)で終わるペアのチェーンという意味で使います。  

<img src="2.4.png" width="75%">

<div style="text-align: center;">図2.4:ペアの鎖として表現された列1, 2, 3, 4</div>

上記のようなリストは、入れ子のconsによって作られます。

In [3]:
; ここで使用している処理系ではnilが使えないため、
; nilと等価である '() を使用しています。
(cons 1 (cons 2 (cons 3 (cons 4 '()))))

(1 2 3 4)

上記の入れ子のconsは、list手続きで作成することが可能です。  
つまり、入れ子のconsとlist手続きで作成されるリストは等価です。

In [4]:
(list 1 2 3 4)

(1 2 3 4)

In [5]:
; クォートを使った以下の書き方も等価です。
; 後で動作確認します。
'(1 2 3 4)

(1 2 3 4)

式 (list 1 2 3 4) と、その式を評価した結果として得られるリスト (1 2 3 4)を混同しないように気をつけてください。  
式(1 2 3 4)を評価 しようとすると、インタプリタは⼿続き1を引数2, 3, 4に適⽤しようとして、 エラーを起こします。 

In [6]:
(define one-through-four (list 1 2 3 4))
one-through-four

(1 2 3 4)

In [7]:
; 先頭の取り出し
(car one-through-four)

1

In [8]:
; 2番目以降の取り出し(ペアの後尾を返す)
(cdr one-through-four)

(2 3 4)

<img src="2.2-cdr.png" width="75%">

<div style="text-align: center;">リストに対するcdrの動作</div>

In [9]:
(car (cdr one-through-four))

2

In [10]:
; 10とリスト(1 2 3 4)をconsでペアにすると、
; (10 1 2 3 4)という新しいリストが生成できます。
(cons 10 one-through-four)

(10 1 2 3 4)

<img src="2.2-cons1.png" width="75%">

<div style="text-align: center;">リストに対するconsの動作</div>

In [11]:
; リスト(1 2 3 4)と5をconsでペアにすると、
; consの入れ子の形にならないので、出力するとリストの形では表示されません。
(cons one-through-four 5)

((1 2 3 4) . 5)

<img src="2.2-cons2.png" width="75%">

<div style="text-align: center;">リストに対するconsの動作</div>

リストから2番目の要素を取り出す方法としてcadrが用意されています。

    (cadr ⟨arg⟩) = (car (cdr ⟨arg⟩))

同様にリストから3番目の要素を取り出す方法としてcaddrが用意されています。

    (caddr ⟨arg⟩) = (car (cdr (cdr ⟨arg⟩)))
    
dの数が取り出す要素の順番に対応しています。

In [12]:
; 注釈にあるcadr手続き
(cadr one-through-four)

2

In [13]:
; 注釈にあるcaddr手続き
(caddr one-through-four)

3

#### リスト演算

ここではリストに対する操作について説明します。  
cdrダウンとconsアップというテクニックでリストの操作を実装します。  
（リストに対する操作は、自分で実装する場合、  
  リストの構造から考えて基本的には、cdrダウンとconsアップしかないと思われます）

In [14]:
; リストからn番目(0オリジン)の要素を取り出します。
; n-1番目までcdrダウンし、目的の位置になったらそのときの要素を返します。
(define (list-ref items n)
  (if (= n 0) (car items)
      (list-ref (cdr items) (- n 1)))
  )

; 動作確認
(define squares (list 1 4 9 16 25))
(list-ref squares 3)

16

In [15]:
; リストの長さ(要素数)を返します。
; 再帰プロセス版
(define (length items)
  (if (null? items) 0
      (+ 1 (length (cdr items))))
  )

; 動作確認
(define odds (list 1 3 5 7))
(length odds)

4

In [16]:
; リストの長さ(要素数)を返す。
; 反復プロセス版
(define (length items)
  (define (length-iter a count)
    (if (null? a) count
        (length-iter (cdr a) (+ 1 count)))
    )
  (length-iter items 0)
  )

; 動作確認
(length odds)

4

In [17]:
; 2つのリストを接続したリストを返します。
; append手続きはデフォルトで定義済みの操作です。
(append squares odds)

(1 4 9 16 25 1 3 5 7)

In [18]:
; 入れ替えた場合
(append odds squares)

(1 3 5 7 1 4 9 16 25)

In [19]:
; appendの実装例
; consアップの例になっています。
; consアップとcdrダウンと組み合わせて新しいリストを構築します。
(define (append list1 list2)
  (if (null? list1) list2
      (cons (car list1) (append (cdr list1) list2)))
  )

In [20]:
(append squares odds)

(1 4 9 16 25 1 3 5 7)

##### 練習問題

- [練習問題2.17 last-pair](../exercises/2.17.ipynb)
- [練習問題2.18 reverse](../exercises/2.18.ipynb)
- [練習問題2.19 リストを使った両替の組み合わせパターン](../exercises/2.19.ipynb)
- [練習問題2.20 ドット記法(可変引数)](../exercises/2.20.ipynb)

#### リストに対するマップ

ここでリストの各数値に対して、演算（操作：写像）を行い、  
その結果をリストとして出力する処理を考えます。

In [21]:
(define (scale-list items factor)
  (if (null? items) '()
      (cons (* (car items) factor) (scale-list (cdr items) factor)))
  )
(scale-list (list 1 2 3 4 5) 10)

(10 20 30 40 50)

<img src="2.2-list-map.png" width="75%">

<div style="text-align: center;">リストに対するマップの動作</div>

1.3節であった、高階手続き（手続きを引数にとる手続き）の考え方に基づくと、  
入力データとなるリストと写像に当たる手続きを引数にとり、  
写像を施した結果をリストにするという処理を一般化することができます。  
この手続きはmapと呼ばれ、schemeで標準的に実装されており、  
より高いレベルの抽象化になっています。  
map手続きは、リスト の要素をどうやって取り出して結合していくかという  
細かいところから切り離しています。

In [24]:
; mapの実装例
(define (map proc items)
  (if (null? items) '()
      (cons (proc (car items)) (map proc (cdr items)))))

In [25]:
; mapの動作確認1
(map abs (list -10 2.5 -11.6 17))

(10 2.5 11.6 17)

In [26]:
; mapの動作確認2
(map (lambda (x) (* x x)) (list 1 2 3 4))

(1 4 9 16)

In [27]:
; scale-listをmapで書き直した場合
(define (scale-list items factor) 
  (map (lambda (x) (* x factor)) items)
  )
(scale-list (list 1 2 3 4 5) 10)

(10 20 30 40 50)

以下は、注釈にある、schemeで定義済みのmapの動作確認です。  
このmap手続きは上記の実装例のmapより汎用的な動作となっています。  
同じ要素数のリストを複数とり、それに対して処理を行います。

$$
f:\mathbb{R} \times \mathbb{R} \times \mathbb{R} \rightarrow \mathbb{R} \\
$$
$$
(x, y, z) \mapsto (x + y + z)
$$

In [22]:
(map + (list 1 2 3) (list 40 50 60) (list 700 800 900))

(741 852 963)

$$
g:\mathbb{R} \times \mathbb{R} \rightarrow \mathbb{R} \\
$$
$$
(x, y) \mapsto (x + 2y)
$$

In [23]:
(map (lambda (x y) (+ x (* 2 y)))
     (list 1 2 3)
     (list 4 5 6))

(9 12 15)

「2.3.1クォート」で紹介される、以下のクォートによる記法によるリストの記述も、  
list手続きの表現と等価です。  

In [28]:
(append '(1 2 3) '(2 3 4))

(1 2 3 2 3 4)

In [29]:
(scale-list '(1 2 3 4 5) 10)

(10 20 30 40 50)

練習問題

- [練習問題2.21 square-list mapの使用方法](../exercises/2.21.ipynb)
- [練習問題2.22 square-listの反復プロセス](../exercises/2.22.ipynb)
- [練習問題2.23 for-each](../exercises/2.23.ipynb)